In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import os
import pandas as pd

# Конфигурация
DATA_DIR = "../data"
MODELS_PREDICTION_DIR = os.path.join(DATA_DIR, "models_predictions")
MODEL_PREFIX = "model_"  # Префикс для файлов моделей

def combine_results(data_dir, model_prefix):
    """
    Объединяет результаты всех моделей в один DataFrame.

    Параметры:
        data_dir (str): Путь к папке с данными.
        model_prefix (str): Префикс для файлов моделей.

    Возвращает:
        pd.DataFrame: Объединенный DataFrame с результатами.
    """
    result_df = pd.DataFrame()

    # Поиск всех файлов с указанным префиксом
    file_pattern = os.path.join(data_dir, f"{model_prefix}*_return_series.csv")
    filepaths = [f for f in glob.glob(file_pattern) if os.path.isfile(f)]

    if not filepaths:
        print("Файлы с результатами не найдены.")
        return result_df

    for filepath in filepaths:
        try:
            # Извлекаем название модели из имени файла
            filename = os.path.basename(filepath)
            model_name = filename[len(model_prefix):-len("_return_series.csv")].upper()

            # Читаем данные из файла
            df_tmp = pd.read_csv(filepath)

            # Определяем, какой столбец использовать как дату
            if 'date' in df_tmp.columns:
                date_column = 'date'
            elif 'Close time' in df_tmp.columns:
                date_column = 'Close time'
            else:
                print(f"Файл {filepath} не содержит необходимых столбцов с датой. Пропускаем.")
                continue

            # Определяем столбец с доходностью
            if 'daily_return' in df_tmp.columns:
                value_column = 'daily_return'
            elif len(df_tmp.columns) > 1:  # Если есть второй столбец
                value_column = df_tmp.columns[1]  # Берем второй столбец
            else:
                print(f"Файл {filepath} не содержит необходимого столбца с доходностью. Пропускаем.")
                continue

            # Преобразуем столбец с датой в формат datetime и устанавливаем его как индекс
            df_tmp[date_column] = pd.to_datetime(df_tmp[date_column])
            df_tmp.set_index(date_column, inplace=True)

            # Переименовываем столбец с доходностью в название модели
            df_tmp.rename(columns={value_column: model_name}, inplace=True)

            # Добавляем данные в общий DataFrame
            if result_df.empty:
                result_df = df_tmp
            else:
                result_df = result_df.join(df_tmp, how='outer')

        except Exception as e:
            print(f"Ошибка при обработке файла {filepath}: {e}")
            continue

    # Устанавливаем индекс на основе дат
    if not result_df.empty:
        result_df.index = result_df.index.strftime('%Y-%m-%d')

    return result_df


import glob

# Вызов функции для объединения результатов
combined_results = combine_results(
    data_dir=MODELS_PREDICTION_DIR,
    model_prefix=MODEL_PREFIX
).dropna()

# Выводим результат
print(combined_results)

In [ ]:
df = combined_results.cumsum()
df

In [ ]:

df.plot(linewidth=2, figsize=(12,8))
plt.title('Сравнение моделей с бенчмарками по доходности')
plt.ylabel('Return')
plt.grid()
plt.axhline(y=0, color='black')
plt.show()

In [ ]:
print('========== ДОХОДНОСТЬ ==========')
print(df.iloc[-1])

print("\n========== ШАРП ==========")
print(df.iloc[-1] / df.std())

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Конфигурация
DATA_DIR = "../data"
MODELS_PREDICTION_DIR = os.path.join(DATA_DIR, "models_predictions")
MODEL_PREFIX = "model_"  # Префикс для файлов моделей

def plot_hourly_actions(data_dir, model_prefix, resample_period='H'):
    """
    Строит графики действий (долей портфеля) для всех моделей.

    Параметры:
        data_dir (str): Путь к папке с данными.
        model_prefix (str): Префикс для файлов моделей.
        resample_period (str): Период ресемплирования ('H', 'D', 'W').

    Возвращает:
        None
    """
    # Поиск всех файлов с указанным префиксом
    file_pattern = os.path.join(data_dir, f"{model_prefix}*_actions.csv")
    filepaths = [f for f in glob.glob(file_pattern) if os.path.isfile(f)]

    if not filepaths:
        print("Файлы с действиями не найдены.")
        return

    for filepath in filepaths:
        try:
            # Извлекаем название модели из имени файла
            filename = os.path.basename(filepath)
            model_name = filename[len(model_prefix):-len("_actions.csv")].upper()

            # Читаем данные из файла
            actions_df = pd.read_csv(filepath)

            # Убедимся, что столбец 'date' существует
            if 'date' not in actions_df.columns:
                print(f"Файл {filepath} не содержит столбца 'date'. Пропускаем модель {model_name}.")
                continue

            # Преобразуем столбец 'date' в формат datetime и устанавливаем его как индекс
            actions_df['date'] = pd.to_datetime(actions_df['date'])
            actions_df.set_index('date', inplace=True)

            # Ресемплирование данных
            actions_df = actions_df.resample(resample_period).mean()

            # Строим график
            actions_df.plot(figsize=(12, 6), title=f"{resample_period} Actions for {model_name}")
            plt.legend(actions_df.columns, loc='upper left', bbox_to_anchor=(1.0, 1.0))
            plt.grid(axis='y')
            plt.ylabel('Portfolio shares')
            plt.xlabel(f'Rebalancing points ({resample_period})')
            plt.tight_layout()
            plt.show()

        except Exception as e:
            print(f"Ошибка при обработке файла {filepath}: {e}")


import glob

# Вызов функции для построения графиков
plot_hourly_actions(
    data_dir=MODELS_PREDICTION_DIR,
    model_prefix=MODEL_PREFIX,
    resample_period='W'  # 'H' (часовой), 'D' (дневной), 'W' (недельный)
)

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Конфигурация
DATA_DIR = "../data"
MODELS_PREDICTION_DIR = os.path.join(DATA_DIR, "models_predictions")
DRL_METHODS = ['a2c', 'ppo', 'ddpg', 'sac', 'markowitz']

for method in DRL_METHODS:
    filepath = os.path.join(MODELS_PREDICTION_DIR, f"model_{method}_actions.csv")
    try:
        # Чтение данных
        actions_df = pd.read_csv(filepath)
        actions_df['date'] = pd.to_datetime(actions_df['date'])
        actions_df.set_index('date', inplace=True)

        # Ресемплинг по месяцам и усреднение
        actions_df = actions_df.resample('3D').mean()
        actions_df.index = actions_df.index.strftime('%Y-%m')

        # Построение графика
        actions_df.plot(kind='bar', stacked=True, figsize=(12, 6))
        plt.legend(actions_df.columns, loc='upper left', bbox_to_anchor=(1.0, 1.0))
        plt.grid(axis='y')
        plt.title(f"Monthly Actions for {method.upper()}")
        plt.ylabel('Portfolio shares')
        plt.xlabel('Rebalancing points (monthly)')
        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"Ошибка при обработке файла {filepath}: {e}")